In [15]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train_epoch(net,data_loader,loss_fn,optimizer,device):
    size = len(data_loader.dataset)
    net.train()
    for batch,(X,y) in enumerate(data_loader):
        X = X.to(device)
        y = y.to(device)
        y_hat = net(y)
        loss = loss_fn(y,y_hat)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch%50==0:
            loss = loss.item()
            current_batch = (batch+1)*len(X)
            print(f'loss:{loss:>5f},[{current_batch:>5d}/{size:>5d}]')

In [ ]:
def test_epoch(net,data_loader,loss_fn,optimizer,device):
    